In [6]:
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
# from keras_diagram import ascii
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import keras
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score

data_raw = pd.read_csv("creditcard.csv")



In [7]:
# Assign variables x and y corresponding to row data and it's class value
X = data_raw.loc[:, data_raw.columns != 'Class']
y = data_raw.loc[:, data_raw.columns == 'Class']

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def generate_train_test_timeseries(data, test_ratio=0.3):
#     cutoff = math.floor(X.shape[0] *cuttoff_percent) 
#     print cutoff
    total_samples = data.shape[0]
    # splitting index
    cutoff = int(total_samples * (1 - test_ratio))

    
    data.sort_values('Time', inplace=True)
    
    X_train = data.loc[0:cutoff, data.columns != 'Class']
    y_train = data.loc[0:cutoff, data.columns == 'Class']
    X_test = data.loc[cutoff:, data.columns != 'Class']
    y_test = data.loc[cutoff:, data.columns == 'Class']
    
    pipeline = Pipeline([
        ('scaling', StandardScaler()),
    ])
    preprocessor = pipeline.fit(X_train)
    X_train_prp = preprocessor.transform(X_train)
    

    X_test_prp = preprocessor.transform(X_test)
    
    return X_train_prp, y_train, X_test_prp, y_test
    
########################################################################

In [9]:
def reshape_to_batches(a, batch_size):
    # pad with zeros if the length is not divisible by the batch_size
    batch_num = np.ceil((float)(a.shape[0]) / batch_size)
    modulo = batch_num * batch_size - a.shape[0]
    if modulo != 0:
        pad = np.zeros((int(modulo), a.shape[1]))
        a = np.vstack((a, pad))
    return np.array(np.split(a, batch_num))

In [10]:
# a function to reshape batches into the original shape
def _3d_to_2d(arr):
    return arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])

In [11]:
def create_model():
    #import all dependencies
    from keras.layers import Input, Dense, Conv1D
    from keras.models import Model

    seed(2018)
    
    inputs = Input(shape=(100, 30)) # This returns a tensor
    
    # a layer instance is callable on a tensor, and returns a tensor
    conv1 = Conv1D(32, (5), # 32 filters with a window of width 5
    strides=1, # think autoregression
    padding='causal', # forward in time
    )(inputs) # syntax to chain layers: Layer(...)(PreviousLayer)
    
    fc1 = Dense(64, activation='relu')(conv1)
    predictions = Dense(2, activation='softmax')(fc1)
    
    model = Model(inputs=inputs,
    outputs=predictions)
    model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
    
    return model

In [15]:
def time_series_cross_val():
    
    precision = []
    recall = []
    f1score = []
    aucs = []
    
    cv = 0.3
    for i in range(3):
        print('Cross validating epoch ', i)
        X_train, y_train, X_test, y_test = generate_train_test_timeseries(data_raw, test_ratio = cv)

        y_test = keras.utils.to_categorical(y_test)
        y_train = keras.utils.to_categorical(y_train)

        X_train_batch = reshape_to_batches(X_train, 100)
        y_train_batch = reshape_to_batches(y_train, 100)
        y_test_batch = reshape_to_batches(y_test, 100)
        X_test_batch = reshape_to_batches(X_test, 100)

        print X_train_batch.shape
        print X_test_batch.shape
        print y_train_batch.shape
        print y_test_batch.shape

        model = create_model()
        model.fit(X_train_batch, y_train_batch, epochs=30)
        y_pred = model.predict(X_test_batch)

        # store the raw predictions in a copy
        y_hat = np.copy(y_pred)

        # Classification reporting
        from sklearn.metrics import confusion_matrix, classification_report
        cutt_off_tr = 0.5
        y_pred[np.where(y_pred>=cutt_off_tr)] = 1
        y_pred[np.where(y_pred<cutt_off_tr)]  = 0

        y_pred_2d = _3d_to_2d(y_pred)[:,1]
        y_test_2d = _3d_to_2d(y_test_batch)[:,1]

        print(confusion_matrix(
        _3d_to_2d(y_test_batch)[:, 1],
        _3d_to_2d(y_pred)[:, 1]))

        prfs = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[1])

        precision.append(prfs[0])
        recall.append(prfs[1])
        f1score.append(prfs[2])
        aucs.append(roc_auc_score(y_test_2d, _3d_to_2d(y_hat)[:, 1]))

        print prfs
        
        print(classification_report(
        y_test_2d,
        y_pred_2d,
        target_names = ["Genuine", "Fraud"],
        digits = 5))
        
        cv = cv - 0.1
        
    
    return [np.mean(f1score),np.mean(precision),np.mean(recall),np.mean(aucs)]

In [16]:
results = time_series_cross_val()

('Cross validating epoch ', 0)
(1994, 100, 30)
(855, 100, 30)
(1994, 100, 2)
(855, 100, 2)
Epoch 1/30
1994/1994 [==============================] - 2s 896us/step - loss: 0.1896 - acc: 0.9458
Epoch 2/30
1994/1994 [==============================] - 1s 578us/step - loss: 0.0170 - acc: 0.9988
Epoch 3/30
1994/1994 [==============================] - 1s 624us/step - loss: 0.0094 - acc: 0.9989
Epoch 4/30
1994/1994 [==============================] - 1s 593us/step - loss: 0.0072 - acc: 0.9990
Epoch 5/30
1994/1994 [==============================] - 1s 603us/step - loss: 0.0059 - acc: 0.9991
Epoch 6/30
1994/1994 [==============================] - 1s 592us/step - loss: 0.0052 - acc: 0.9991
Epoch 7/30
1994/1994 [==============================] - 1s 571us/step - loss: 0.0046 - acc: 0.9992
Epoch 8/30
1994/1994 [==============================] - 1s 575us/step - loss: 0.0043 - acc: 0.9992
Epoch 9/30
1994/1994 [==============================] - 1s 574us/step - loss: 0.0040 - acc: 0.9992
Epoch 10/30
1994/1

2564/2564 [==============================] - 1s 476us/step - loss: 0.0031 - acc: 0.9994
Epoch 14/30
2564/2564 [==============================] - 1s 486us/step - loss: 0.0030 - acc: 0.9994
Epoch 15/30
2564/2564 [==============================] - 1s 501us/step - loss: 0.0031 - acc: 0.9993
Epoch 16/30
2564/2564 [==============================] - 1s 483us/step - loss: 0.0027 - acc: 0.9995
Epoch 17/30
2564/2564 [==============================] - 1s 478us/step - loss: 0.0026 - acc: 0.9994
Epoch 18/30
2564/2564 [==============================] - 1s 517us/step - loss: 0.0025 - acc: 0.9995
Epoch 19/30
2564/2564 [==============================] - 1s 492us/step - loss: 0.0024 - acc: 0.9995
Epoch 20/30
2564/2564 [==============================] - 1s 479us/step - loss: 0.0023 - acc: 0.9995
Epoch 21/30
2564/2564 [==============================] - 1s 508us/step - loss: 0.0022 - acc: 0.9995
Epoch 22/30
2564/2564 [==============================] - 1s 477us/step - loss: 0.0021 - acc: 0.9995
Epoch 23/30


In [17]:
results

[0.6706349206349206, 0.9413105413105414, 0.522716049382716, 0.9524019821436042]